In [ ]:
import tweepy
import os
from google.colab import userdata

consumer_key = userdata.get("TWITTER_CONSUMER_KEY")
consumer_secret = userdata.get("TWITTER_CONSUMER_SECRET")
access_token = userdata.get("TWITTER_ACCESS_TOKEN")
access_token_secret = userdata.get("TWITTER_ACCESS_TOKEN_SECRET")

if not all([consumer_key, consumer_secret, access_token, access_token_secret]):
    raise ValueError("Twitter API credentials not found. Please set TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET, TWITTER_ACCESS_TOKEN, and TWITTER_ACCESS_TOKEN_SECRET in Google Colab userdata.")

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except Exception as e:
    print(f"Error during authentication: {e}")

In [ ]:
@tool
def twitter_search(query, num_tweets=5):
    try:
        tweets = api.search_tweets(q=query, count=num_tweets)
        results = []
        for tweet in tweets:
            results.append(tweet.text)
        return "\n".join(results)
    except Exception as e:
        return f"Error searching Twitter: {e}"

In [ ]:
from crewai import Crew

crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task],
    llm=llm,
    verbose=True,
    tools=[rag_tool, web_search_tool, twitter_search]
)

question = "What are the different types of breast cancer?"  
result = crew.run(question=question)
result